In [21]:
def maybe_download_and_extract(dest_directory, url):
  """
    Download the dataset and extract the data
  """
    
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  file_name = 'cifar-10-binary.tar.gz'
  file_path = os.path.join(dest_directory, file_name)
  # if have not downloaded yet
  if not os.path.exists(file_path):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r%.1f%%' % 
            (float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()  # flush the buffer

    print('>> Downloading %s ...' % file_name)
    file_path, _ = urllib.request.urlretrieve(url, file_path, _progress)
    file_size = os.stat(file_path).st_size
    print('\r>> Total %d bytes' % file_size)
  extracted_dir_path = os.path.join(dest_directory, 'cifar-10-batches-bin')
  if not os.path.exists(extracted_dir_path):
    # Open for reading with gzip compression, then extract all
    tarfile.open(file_path, 'r:gz').extractall(dest_directory)
  print('>> Done')

# download it

In [22]:
import os
import sys
from six.moves import urllib
import tarfile
import tensorflow as tf
import numpy as np
DATA_URL = 'http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'
DEST_DIRECTORY = './dataset/cifar10'
DATA_DIRECTORY = DEST_DIRECTORY + '/cifar-10-batches-bin'
IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32
IMAGE_DEPTH = 3
IMAGE_SIZE_CROPPED = 24
BATCH_SIZE = 128
NUM_CLASSES = 10 
LABEL_BYTES = 1
IMAGE_BYTES = 32 * 32 * 3
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

# The record is the image plus a one-byte label
_RECORD_BYTES = IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_DEPTH + 1
# download it
maybe_download_and_extract(DEST_DIRECTORY, DATA_URL)
training_files = [os.path.join(DATA_DIRECTORY, 'data_batch_%d.bin' % i) for i in range(1,6)]
testing_files = [os.path.join(DATA_DIRECTORY, 'test_batch.bin')]



>> Done


In [23]:
# (5) + (6)
def read_cifar10(filename_queue):
    """ Reads and parses examples from CIFAR10 data files.
    -----
    Args:
        filename_queue: 
            A queue of strings with the filenames to read from.
    Returns:
        An object representing a single example, with the following fields:
        height: 
            number of rows in the result (32)
        width: 
            number of columns in the result (32)
        depth: 
            number of color channels in the result (3)
        key: 
            a scalar string Tensor describing the filename & record number for this example.
        label: 
            an int32 Tensor with the label in the range 0..9.
        image: 
            a [height, width, depth] uint8 Tensor with the image data
    """

    class CIFAR10Record(object):
        pass

    result = CIFAR10Record()
    # CIFAR10 consists of 60000 32x32 'color' images in 10 classes
    label_bytes = 1  # 10 class
    result.height = IMAGE_HEIGHT
    result.width = IMAGE_WIDTH
    result.depth = IMAGE_DEPTH
    image_bytes = result.height * result.width * result.depth
    # bytes of a record: label(1 byte) followed by pixels(3072 bytes)
    record_bytes = label_bytes + image_bytes
    # (5) reader for cifar10 file format
    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    # read a record
    result.key, record_string = reader.read(filename_queue)
    # Convert from a string to a vector of uint8 that is record_bytes long.
    # (6) decoder
    record_uint8 = tf.decode_raw(record_string, tf.uint8)
    # get the label and cast it to int32
    result.label = tf.cast(
      tf.strided_slice(record_uint8, [0], [label_bytes]), tf.int32)
    # [depth, height, width], uint8
    depth_major = tf.reshape(
      tf.strided_slice(record_uint8, [label_bytes],
                       [label_bytes + image_bytes]),
      [result.depth, result.height, result.width])
    # change to [height, width, depth], uint8
    result.image = tf.transpose(depth_major, [1, 2, 0])
    return result

In [56]:
class CNN_Model(object):
    def __init__(self, model_hps):
        self.image_size = model_hps.image_size
        self.batch_size = model_hps.batch_size
        self.num_classes = model_hps.num_classes
        self.num_training_example = model_hps.num_training_example
        self.num_epoch_per_decay = model_hps.num_epoch_per_decay
        self.init_lr = model_hps.init_lr  # initial learn rate
        self.moving_average_decay = model_hps.moving_average_decay
        self.ckpt_dir = model_hps.ckpt_dir

        self.build_model()

    def build_model(self):
        # op for training
        self.global_step = tf.contrib.framework.get_or_create_global_step()

        with tf.variable_scope('model'):
            self.images = tf.placeholder(tf.float32,[self.batch_size, self.image_size, self.image_size, 3]) 
            self.labels = tf.placeholder(tf.int32)

            self.logits = self.inference(self.images)
            self.top_k_op = tf.nn.in_top_k(self.logits, self.labels, 1) 
            self.total_loss = self.loss(self.logits, self.labels)
            self.train_op = self.train(self.total_loss, self.global_step)

    def _variable_on_cpu(self, name, shape, initializer):
        with tf.device('/cpu:0'):
            var = tf.get_variable(name, shape, initializer=initializer, dtype=tf.float32)

        return var

    def _variable_with_weight_decay(self, name, shape, stddev, wd=0.0):
        """ Helper to create an initialized Variable with weight decay.
            Note that the Variable is initialized with a truncated normal 
            distribution. A weight decay is added only if one is specified.
            -----
            Args:
                name: 
                    name of the variable
                shape: 
                    a list of ints
                stddev: 
                    standard deviation of a truncated Gaussian
                wd: 
                    add L2Loss weight decay multiplied by this float. If None, weight
                    decay is not added for this Variable.
            Returns:
                Variable Tensor
        """
        initializer = tf.truncated_normal_initializer(
            stddev=stddev, dtype=tf.float32)
        var = self._variable_on_cpu(name, shape, initializer)
        # deal with weight decay
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
        return var

    def _conv_block(self, inp, scope, kernel_width, kernel_height, inp_channel, out_channel, strides = [1, 1, 1, 1], padding='SAME'):
        with tf.variable_scope(scope) as scope:
            kernel = self._variable_with_weight_decay('weights', [kernel_width, kernel_width, inp_channel, out_channel], 5e-2)
            biases = self._variable_on_cpu('bias', [out_channel], tf.constant_initializer(0.0))

            conv = tf.nn.conv2d(inp, kernel, strides=strides, padding=padding)
            pre_activation = tf.nn.bias_add(conv, biases)
            return tf.nn.relu(pre_activation, name=scope.name)

    def _fully_connected_layer(self, inp, scope, in_dim, out_dim, relu = True):
        with tf.variable_scope(scope) as scope:
            weights = self._variable_with_weight_decay('weights', [in_dim, out_dim], 0.04, 0.004)
            biases = self._variable_on_cpu('biases', [out_dim], tf.constant_initializer(0.1))
            if relu:
                return tf.nn.relu(tf.matmul(inp, weights) + biases, name=scope.name)
            else:
                return tf.matmul(inp, weights) + biases

    def inference(self, images):
        """ build the model
            -----
            Args:
                images with shape [batch_size,24,24,3]
            Return:
                logits with shape [batch_size,10]
        """
        conv_1 = self._conv_block(images, 'conv_1', 5, 5, 3, 64)
        # pool_1
        pool_1 = tf.nn.max_pool(conv_1,ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool_1')
        # norm_1 (local_response_normalization)
        norm_1 = tf.nn.lrn(pool_1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm_1')

        # conv2
        conv_2 = self._conv_block(norm_1, 'conv_2', 5, 5, 64, 64)
        # norm2
        norm_2 = tf.nn.lrn(conv_2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm_2')
        # pool2
        pool_2 = tf.nn.max_pool(norm_2,ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool_2')

        # Flatten feature maps before fully connected layers
        flat_features = tf.reshape(pool_2, [self.batch_size, -1])
        dim = flat_features.get_shape()[1].value
        # FC_1 (fully-connected layer)
        fc_1 = self._fully_connected_layer(flat_features, 'fc1', dim, 384)

        # FC_2
        fc_2 = self._fully_connected_layer(fc_1, 'fc2', 384, 192)

        logits = self._fully_connected_layer(fc_2, 'softmax_linear', 192, self.num_classes, relu = False)
        return logits

    def loss(self, logits, labels):
        '''calculate the loss'''
        labels = tf.cast(labels, tf.int64)
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=labels, logits=logits, name='cross_entropy_per_example')
        cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
        tf.add_to_collection('losses', cross_entropy_mean)
        # The total loss is defined as the cross entropy loss plus all of the weight
        # decay terms (L2 loss).
        return tf.add_n(tf.get_collection('losses'), name='total_loss')

    def train(self, total_loss, global_step):
        '''Return training operation of one step'''
        num_batches_per_epoch = self.num_training_example / self.batch_size
        decay_steps = int(num_batches_per_epoch * self.num_epoch_per_decay)
        # Decay the learning rate exponentially based on the number of steps.
        lr = tf.train.exponential_decay(
            self.init_lr, global_step, decay_steps, decay_rate=0.1, staircase=True)
        opt = tf.train.GradientDescentOptimizer(lr)
        grads = opt.compute_gradients(total_loss)
        apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)
        # Track the moving averages of all trainable variables.
        # This step just records the moving average weights but not uses them
        ema = tf.train.ExponentialMovingAverage(self.moving_average_decay,
                                                global_step)
        self.ema = ema
        variables_averages_op = ema.apply(tf.trainable_variables())
        with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
            train_op = tf.no_op(name='train')
        return train_op

In [26]:
def run_training(model, data_train , num_epoch):
    saver = tf.train.Saver()
        
    with tf.Session() as sess:
        sess.run(iterator_train.initializer)
        
        ckpt = tf.train.get_checkpoint_state(model.ckpt_dir)
        if (ckpt and ckpt.model_checkpoint_path):
            saver.restore(sess, ckpt.model_checkpoint_path)
            # assume the name of checkpoint is like '.../model.ckpt-1000'
            gs = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
            sess.run(tf.assign(model.global_step, gs))
        else:
          # no checkpoint found
            print('no ckpt, init global variable...')
            sess.run(tf.global_variables_initializer())

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        model.loss_each_epoch = []

        num_batch_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN//model.batch_size
        #start training
        for i in range(num_epoch):
            _loss = []
            print('epoch:',i)
            for _ in range(num_batch_per_epoch):
                print('.',end='')
                images, labels = sess.run(data_train)
                l, _ = sess.run([model.total_loss, model.train_op], feed_dict = {model.images:images, model.labels:labels})
                _loss.append(l)
            loss_this_epoch = np.sum(_loss)
            gs = model.global_step.eval()
            print('loss of epoch %d: %f' % (gs / num_batch_per_epoch, loss_this_epoch))
            model.loss_each_epoch.append(loss_this_epoch)
            saver.save(sess, model.ckpt_dir + 'model.ckpt', global_step=gs)
        coord.request_stop()
        coord.join(threads)
    print('Done training %d epochs' %num_epoch)

In [27]:
def run_testing(model, data_test):
    variables_to_restore = model.ema.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)
    with tf.Session() as sess:
        # Restore variables from disk.
        ckpt = tf.train.get_checkpoint_state(model.ckpt_dir)

        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            num_iter = NUM_EXAMPLES_PER_EPOCH_FOR_EVAL // model.batch_size
            total_sample_count = num_iter * model.batch_size
            true_count = 0
            for _ in range(num_iter):
                images, labels = sess.run(data_test) 

                predictions = sess.run(model.top_k_op, feed_dict = {model.images:images, model.labels:labels})
                true_count += np.sum(predictions)
            print('Accurarcy: %d/%d = %f' % (true_count, total_sample_count,
                                             true_count / total_sample_count))
            coord.request_stop()
            coord.join(threads)
        else:
            print('train first')

In [54]:
model_hps_cifar = tf.contrib.training.HParams(
  image_size = IMAGE_SIZE_CROPPED,
  batch_size = BATCH_SIZE,
  num_classes = NUM_CLASSES,
  num_training_example = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN,
  num_epoch_per_decay = 350.0,
  init_lr = 0.1,
  moving_average_decay = 0.9999,
  ckpt_dir = './model/'
)

In [82]:
def cifar10_record_distort_parser(record):

    record_bytes = LABEL_BYTES + IMAGE_BYTES
    record = tf.decode_raw(record, tf.uint8)
    label  = tf.cast(record[0], tf.int32)
    
    image = tf.reshape(record[1:record_bytes]
                       , [IMAGE_DEPTH, IMAGE_HEIGHT, IMAGE_WIDTH])
    
    reshaped_image = tf.cast(tf.transpose(image, [1, 2, 0]), tf.float32)
    distorted_image = tf.random_crop(reshaped_image
                                     , [IMAGE_SIZE_CROPPED, IMAGE_SIZE_CROPPED, 3])
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
    distorted_image = tf.image.per_image_standardization(distorted_image)
    
    return label, distorted_image
    


def cifar10_record_crop_parser(record):

    record_bytes = LABEL_BYTES + IMAGE_BYTES
    record = tf.decode_raw(record, tf.uint8)
    label  = tf.cast(record[0], tf.int32)
    
    image = tf.reshape(record[1:record_bytes]
                       , [IMAGE_DEPTH, IMAGE_HEIGHT, IMAGE_WIDTH])
    
    reshaped_image = tf.cast(tf.transpose(image, [1, 2, 0]), tf.float32)
    cropped_image = tf.random_crop(reshaped_image
                                     , [IMAGE_SIZE_CROPPED, IMAGE_SIZE_CROPPED, 3])
    cropped_image = tf.image.per_image_standardization(cropped_image)
    
    return label, cropped_image


def cifar10_iterator(filenames, batch_size, cifar10_record_parser):

    record_bytes = LABEL_BYTES + IMAGE_BYTES
    dataset = tf.data.FixedLengthRecordDataset(filenames, record_bytes)
    dataset = dataset.map(cifar10_record_parser)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(10)
    
    iterator = dataset.make_initializable_iterator()
    

    return iterator, dataset.output_types, dataset.output_shapes


tf.reset_default_graph()

training_files = [os.path.join(DATA_DIRECTORY, 'data_batch_%d.bin' % i) for i in range(1, 6)]
testing_files = [os.path.join(DATA_DIRECTORY, 'test_batch.bin')]

filenames_train = tf.constant(training_files)
filenames_test = tf.constant(testing_files)

iterator_train, types, shapes = cifar10_iterator(filenames_train, BATCH_SIZE, cifar10_record_distort_parser)
iterator_test, _, _ = cifar10_iterator(filenames_test, BATCH_SIZE, cifar10_record_crop_parser)

next_batch = iterator_train.get_next()

# use to handle training and testing
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, types, shapes)
labels_images_pairs = iterator.get_next()


# CNN model
model = CNN_Model(model_hps_cifar)


print('done')

done


In [86]:
%%time

# train
global_step = tf.contrib.framework.get_or_create_global_step()



from datetime import datetime

from tqdm import tqdm_notebook as tqdm
NUM_EPOCH = 10
NUM_BATCH_PER_EPOCH = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN // BATCH_SIZE
ckpt_dir = './model/'

# config = tf.ConfigProto(allow_soft_placement=True, gpu_options=tf.GPUOptions(allow_growth=True))

# train
saver = tf.train.Saver()

# with tf.Session(config=config) as sess:
with tf.Session() as sess:
    
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    
    if (ckpt and ckpt.model_checkpoint_path):
        saver.restore(sess, ckpt.model_checkpoint_path)
        # assume the name of checkpoint is like '.../model.ckpt-1000'
        gs = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
        sess.run(tf.assign(global_step, gs))
    else:
        # no checkpoint found
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

        sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    loss = []
    training_handle = sess.run(iterator_train.string_handle())
    
    print("{}: Start training.".format(datetime.now()))
        
    for i in range(NUM_EPOCH):
        _loss = []
        sess.run(iterator_train.initializer)

        for _ in tqdm(range(NUM_BATCH_PER_EPOCH)):
            lbl, img = sess.run(next_batch)            
            l, _ = sess.run([model.total_loss, model.train_op], feed_dict={model.images: img, model.labels: lbl})


            
            _loss.append(l)
        loss_this_epoch = np.sum(_loss)
        gs = global_step.eval()
        print('{}: Loss of epoch {}: {}'.format(datetime.now(), gs / NUM_BATCH_PER_EPOCH, loss_this_epoch))
        loss.append(loss_this_epoch)
        saver.save(sess, ckpt_dir + 'model.ckpt', global_step=gs)
    coord.request_stop()
    coord.join(threads)
  
print("{}: Done training.".format(datetime.now()))



INFO:tensorflow:Restoring parameters from ./model/model.ckpt-780
2018-11-03 00:03:01.848300: Start training.


2018-11-03 00:08:38.174215: Loss of epoch 3.0: 956.37109375


2018-11-03 00:14:14.563623: Loss of epoch 4.0: 803.8801879882812


2018-11-03 00:19:51.099676: Loss of epoch 5.0: 688.476806640625


2018-11-03 00:25:28.512447: Loss of epoch 6.0: 606.7338256835938


...

2018-11-03 00:36:55.817455: Loss of epoch 8.0: 503.2664794921875


2018-11-03 00:42:35.797119: Loss of epoch 9.0: 469.2889404296875


2018-11-03 00:48:16.708131: Loss of epoch 10.0: 445.47125244140625


2018-11-03 00:53:57.056173: Loss of epoch 11.0: 424.71630859375


2018-11-03 00:59:36.948213: Loss of epoch 12.0: 408.9622497558594
2018-11-03 00:59:37.156214: Done training.
Wall time: 56min 35s


In [89]:
%%time

next_test = iterator_test.get_next()
variables_to_restore = model.ema.variables_to_restore()
saver = tf.train.Saver(variables_to_restore)
with tf.Session() as sess:
    # Restore variables from disk.
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        num_iter = NUM_EXAMPLES_PER_EPOCH_FOR_EVAL // BATCH_SIZE
        total_sample_count = num_iter * BATCH_SIZE
        true_count = 0
        sess.run(iterator_test.initializer)
        for _ in tqdm(range(num_iter)):
            lbl, img = sess.run(next_test)
            predictions = sess.run(model.top_k_op, feed_dict={model.images: img, model.labels: lbl})
            true_count += np.sum(predictions)
        print('{}: Accurarcy: {}/{} = {}'.format(datetime.now(), true_count, total_sample_count,
                                     true_count / total_sample_count))
        coord.request_stop()
        coord.join(threads)
    else:
        print("{}: No model existed.".format(datetime.now()))

INFO:tensorflow:Restoring parameters from ./model/model.ckpt-4680


2018-11-03 01:00:41.247605: Accurarcy: 7317/9984 = 0.7328725961538461
Wall time: 20.9 s
